# First steps with the ``pyam`` package 

## *An open-source Python package for IAM scenario analysis and visualization*

<img style="float: right; height: 100px; margin-top: 10px;" src="_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="_static/IAMC_logo.jpg">

## Scope and feature overview

The ``pyam`` package provides a range of diagnostic tools and functions  
for analyzing and working with scenario data following the IAMC template format.
A comprehensive documentation of the package is available
at [pyam-iamc.readthedocs.io](http://pyam-iamc.readthedocs.io)

An illustrative example of the IAMC template is shown below;
see [data.ene.iiasa.ac.at/database/](http://data.ene.iiasa.ac.at/database/) for more information.


| **Model**           | **Scenario**  | **Region** | **Variable**   | **Unit** | **2005** | **2010** | **2015** |
|---------------------|---------------|------------|----------------|----------|----------|----------|----------|
| MESSAGE V.4         | AMPERE3-Base  | World      | Primary Energy | EJ/y     | 454.5    |	479.6    | ...      |
| ...                 | ...           | ...        | ...            | ...      | ...      | ...      | ...      |

This notebook illustrates some basic functionality of the ``pyam`` package
and the ``IamDataFrame`` class:

1. Importing timeseries data from `xlsx` or `csv` files.
2. Listing models, scenarios and variables included in the data.
3. Display of timeseries data 
   as [pd.DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).
4. Visualization tools for timeseries data using the [matplotlib](https://matplotlib.org/) package.
5. Evaluating the model data and executing a range of diagnostic checks for identifying outliers.
6. Categorization of scenarios according to timeseries data values or checks on required variables.
7. Exporting data to `xlsx` using the IAMC template.


## Tutorial data

The timeseries data used in this tutorial is a partial snapshot of the scenario database 
compiled for the IPCC's Fifth Assessment Report (AR5):

> Krey V., O. Masera, G. Blanford, T. Bruckner, R. Cooke, K. Fisher-Vanden, H. Haberl, E. Hertwich, E. Kriegler, D. Mueller, S. Paltsev, L. Price, S. Schlömer, D. Ürge-Vorsatz, D. van Vuuren, and T. Zwickel, 2014: *Annex II: Metrics & Methodology*.   

> In: *Climate Change 2014: Mitigation of Climate Change. Contribution of Working Group III to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change* [Edenhofer, O., R. Pichs-Madruga, Y. Sokona, E. Farahani, S. Kadner, K. Seyboth, A. Adler, I. Baum, S. Brunner, P. Eickemeier, B. Kriemann, J. Savolainen, S. Schlömer, C. von Stechow, T. Zwickel and J.C. Minx (eds.)]. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA. [Link](https://www.ipcc.ch/report/ar5/wg3/)

The complete database is publicly available at [tntcat.iiasa.ac.at/AR5DB/](https://tntcat.iiasa.ac.at/AR5DB/).

<img style="float: right; height: 100px;" src="_static/AMPERE-Logo.png">
<img style="float: right; height: 40px; margin-top: 50px; margin-right: 20px;" src="_static/EMF-Logo_v2.1.png">

The data snapshot used for this tutorial consists of selected data from two model intercomparison projects:

 - Energy Modeling Forum Round 27 
   ([EMF27](https://emf.stanford.edu/projects/emf-27-global-model-comparison-exercise)),
   see the Special Issue in [Climatic Change 3-4, 2014](https://link.springer.com/journal/10584/123/3/page/1).
   
 - EU FP7 project [AMPERE](https://tntcat.iiasa.ac.at/AMPEREDB/), 
   see the following scientific publications:
   
   > - Riahi, K., et al. (2015). "Locked into Copenhagen pledges — Implications of short-term emission targets 
   >   for the cost and feasibility of long-term climate goals." 
   >   *Technological Forecasting and Social Change* 90(Part A): 8-23.  
   >   [DOI: 10.1016/j.techfore.2013.09.016](https://doi.org/10.1016/j.techfore.2013.09.016)
   
   > - Kriegler, E., et al. (2015). "Making or breaking climate targets: The AMPERE study on 
   >   staged accession scenarios for climate policy."
   >   *Technological Forecasting and Social Change* 90(Part A): 24-44.  
   >   [DOI: 10.1016/j.techfore.2013.09.021](https://doi.org/10.1016/j.techfore.2013.09.021)

<div style="text-align: center; padding: 10px; border: 2px solid red; width: 700px">
*The data used in this tutorial is ONLY a partial snapshot of the IPCC AR5 scenario database!*  
*This tutorial is only intended for an illustration of the pyam package.*
</div>

## Import package and load data from the AR5 tutorial snapshot

We import the snapshot timeseries data from the file ``tutorial_AR5_data.csv`` in the ``tutorial`` folder.

As a first step, we show lists of all models, scenarios, regions, and the variables and units included in the snapshot.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyam

%matplotlib inline

In [ ]:
df = pyam.IamDataFrame(data='tutorial_AR5_data.csv', encoding='utf-8')

In [ ]:
df.models()

In [ ]:
df.scenarios()

In [ ]:
df.regions()

In [ ]:
df.variables(include_units=True)

## Tutorial on data filtering
A selection of the timeseries data  of an ``IamDataFrame`` can be obtained by applying the ``filter()`` funtion,
which takes a dictionary of filter criteria as argument.
The function `filter()` returns a filtered clone of the ``IamDataFrame``.

### Filtering by model names, scenarios and regions

The feature for filtering by **model, scenario or region** 
are implemented using exact string matching, where ``*`` can be used as a wildcard:

> Applying the keyword argument filter ``model='MESSAGE'`` to the ``IamDataFrame``
  will return an empty array.

> Filtering for ``model='MESSAGE*'`` will return all scenarios from the *"MESSAGE"* family,
> identified by the model name "MESSAGE" and a version identifier.

In [ ]:
df.filter(model='MESSAGE').models()

In [ ]:
df.filter(model='MESSAGE*')[['model', 'scenario']].drop_duplicates()

Using keyword `keep=False` allows you to discard everything that is found by the filter rather than keeping it

In [ ]:
df.filter(region="World", keep=False).regions()

In [ ]:
df.filter(region="World", keep=True).regions()

### Filtering by variables and hierarchy levels

Filtering for **variable** strings works in an identical way as above,
with ``*`` available as a wildcard.

> Filtering for _Primary Energy_ will return only exactly those data.

> Filtering for _Primary Energy|*_ will return all sub-categories of 
> primary-energy level (and only the sub-categories).

In additon, IAM variables can be filtered by the **level**,
i.e., the "depth" of the variable in a hierarchical reading of the string separated by `'|'`.
That is, the variable _Primary Energy_ has level 0, while _Primary Energy|Coal_ has level 1.
Filtering by both **variables** and **level** will search for the hierarchical depth 
_following the variable string_, so filter arguments ``'variable': 'Primary Energy|*'`` and ``'level': 0``
will return all variables immediately below ``'Primary Energy'``.
Filtering by **level** only will return all variables at that depth.

To illustrate the functionality of the filters, we first show all sub-categories of the ``'Emissions'`` variable.  
Then, we reduce variables to only one hierarchical levels below ``'Emissions|'``; the list returned by the function call will include only ``'Emissions|CO2|Fossil Fuels and Industry'``, because the level argument (by default) only shows variables at exactly the hierarchical levels below ``'Emissions|...'``.

The third example illustrates another use case of the level argument - filtering by `'1-'` instead of `1` will return all variables *up to* the specified depth.

The last cell shows how to filter only by hierarchical level, without providing a variable string.
The function returns all variables that are at the top hierarchical level (i.e., ``'Primary Energy'``) and those at the first sub-category level.
Keep in mind that there are no variables ``'Emissions'`` or ``'Price'`` (no top level).

In [ ]:
df.filter(variable='Emissions|*').variables()

In [ ]:
df.filter(variable='Emissions|*', level=1).variables()

In [ ]:
df.filter(variable='Emissions|*', level='1-').variables()

In [ ]:
df.filter(level='1-').variables()

### Filtering by year

Filtering for **years** can be done by integer number, a list of integers, or the Python class ``range``.  
Note that the last year of a range is not included, so ``range(2010,2015)``
is interpreted as ``[2010, 2011, 2012, 2013, 2014]``.

## Getting help

When in doubt, you can look at the help for any function by appending it with a ``?``.

In [ ]:
df.filter?

## Displaying timeseries data

As a next step, we want to view a selection of the data in the tutorial snapshot using the IAMC standard.
The `timeseries()` function returns the data in the standard IAMC format as a `pd.DataFrame`.

In [ ]:
(df
 .filter(scenario='AMPERE3-450', variable='Primary Energy|Coal', region='World')
 .timeseries()
)

For displaying data in a different format, the class ``IamDataFrame`` has a wrapper of the ``pd.DataFrame.pivot_table()`` function. It allows to flexibly specify the columns and rows.
The function automatically aggregates by summation or counting (specified by the parameter `aggfunc`) 
over all timeseries data identifiers ('model', 'scenario', 'variable', 'region', 'unit', 'year')
which are not used as `index` or `columns`.

In the example below, the filter of the timeseries data is set for all subcategories of 'Primary Energy', 
which are then summed up in the displayed table.

In [ ]:
(df
 .filter(variable='Primary Energy', region='World')
 .pivot_table(index=['year'], columns=['scenario'], values='value', aggfunc='sum')
)

If you are familiar with the `python` package `pandas`, you can use many of usual functions directly on the ``IamDataFrame``. The function ``head()``, for example, will show the first `n` rows of the data in long form 
(columns are in year/value format).

In [ ]:
df.head()

## Visualization of timeseries

This section provides one illustrative example of the plotting features of the ``pyam`` package. Please see the [plotting gallery](https://pyam-iamc.readthedocs.io/en/latest/examples/index.html) for more examples on plotting with `pyam`.

In [ ]:
df.filter(variable='Emissions|CO2', region='World').line_plot(legend=False)

## Validation and diagnostic assessment of timeseries data

When analyzing scenario results, it is often useful to check whether certain timeseries exist or the values are within a specific range. For example, it may make sense to ensure that reported data for historical periods are close to established reference data or that near-term developments are reasonable.

The following section provides three illustrations:
1. Check whether a timeseries `'Primary Energy'` exists in each scenario (in at least one year).
2. Check for every scenario whether the value for `'Primary Energy'` at the global level exceeds 515 EJ/y 
   in the reference year 2010
   (the value must satisfy an upper bound of 515 EJ/y in this notation).
3. Check for every scenario from the `AMPERE` project
   whether the value for `'Primary Energy|Coal'` exceeds 400 EJ/y in mid-century.

The `validate()` function performs the checks on models and scenarios.

The ``criteria`` argument can specify a valid range by an upper and lower bound (``up``, ``lo``) for a variable and a subset of years to which the validation is applied - all scenarios with a value in at least one year outside that range are considered to *not satisfy* the validation.

By setting the argument ``exclude=True``, all scenarios failing the validation will be categorized as ``exclude`` in the metadata. This allows to remove these scenarios from subsequent analysis or figures.

In [ ]:
df.require_variable(variable='Primary Energy')

In [ ]:
df.validate(criteria={'Primary Energy': {'up': 515, 'year': 2010}})

In [ ]:
pyam.validate(df.filter(region='World', scenario='AMPERE*'), 
              criteria={'Primary Energy|Coal': {'up': 400, 'year': 2050}}
)

## Categorization of scenarios by timeseries characteristics

It is often useful to apply categorization to classes of scenarios according to specific characteristics of the timeseries data. In the following example, we use the temperature change assessment by MAGICC 6 to group scenarios by the median global warming by the end of the century (year 2100).

We proceed in the following steps:

0. Plot the timeseries data of the variable that we want to use. 
   This provides some insights on useful thresholds for the categorization.
0. Use the function ``categorize()`` to apply a categorization (and colour code for later use) 
   to all scenarios that satisfy a number of specific criteria.
0. Use the categorization of scenarios for analysis of other timeseries data.

In [ ]:
v = 'Temperature|Global Mean|MAGICC6|MED'
df.filter(region='World', variable=v).line_plot(legend=False)

We now use the categorization feature of the ``pyam`` package to group scenarios by temperature outcome by the end of the century.

The first cell sets the ``'Temperature'`` categorization to the default `"uncategorized"`.
This is not necessary per se (setting a meta column via the categorization will mark all non-assigned rows as `"uncategorized"` (if the value is a string) or `np.nan`.
Still, having this cell may be helpful in this tutorial if you are going back and forth between cells to reset the assignment.

The function `categorize()` takes `color` and similar arguments, which can then be used by the plotting library.

In [ ]:
df.set_meta(meta='uncategorized', name='Temperature')

In [ ]:
df.categorize(
    'Temperature', 'Below 1.6C',
    criteria={v: {'up': 1.6, 'year': 2100}},
    color='cornflowerblue'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 2.0C',
    criteria={'Temperature|Global Mean|MAGICC6|MED': {'up': 2.0, 'lo': 1.6, 'year': 2100}},
    color='forestgreen'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 2.5C',
    criteria={v: {'up': 2.5, 'lo': 2.0, 'year': 2100}},
    color='gold'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 3.5C',
     criteria={v: {'up': 3.5, 'lo': 2.5, 'year': 2100}},
     color='firebrick'
)

In [ ]:
df.categorize(
    'Temperature', 'Above 3.5C',
    criteria={v: {'lo': 3.5, 'year': 2100}},
    color='magenta'
)

Two models included in the snapshot have not been assessed by MAGICC6 regarding their long-term climate and warming impact. Therefore, the timeseries ``'Temperature|Global Mean|MAGICC6|MED'`` does not exist, and they have not been categorized.

In [ ]:
df.require_variable(variable=v, exclude_on_fail=False)

Now, we again display the median global temperature increase for all scenarios, but we use the colouring by category to illustrate the common charateristics across scenarios.

In [ ]:
df.filter(variable=v).line_plot(color='Temperature', legend=True)

As a last step, we display the aggregate CO2 emissions by category. This allows to highlight alternative pathways within the same category. 

In this step, we also export this figure as a png using the option ``savefig``. The figure will be saved in the tutorials folder.

In [ ]:
fig, ax = plt.subplots()
(df
 .filter(variable='Emissions|CO2', region='World')
 .line_plot(ax=ax, color='Temperature', legend=True)
)
fig.savefig('co2_emissions.png')

## Using metadata indicators for further scenario characterization & diagnostics

In the example above, we classified scenarios by their end-of-century warming. However, for some research questions, it may be of interest to know the maximum temperature over the course of the century (a.k.a. peak warming).

In this section, we illustrate two ways to add quantitative metadata indicators.
First, we add indicators derived directly from the data in the `IamDataFrame`.

In [ ]:
eoc = 'End-of-century-temperature'
df.set_meta_from_data(name='End-of-century-temperature', variable=v, year=2100)

If the filter arguments passed to `set_meta_from_data()` do not yield a unique value,
we can pass a `method` to aggregate.

In [ ]:
peak = 'Peak-temperature'
df.set_meta_from_data(name='Peak-temperature', variable=v, method=np.max)

The second method to define metadata indicators can take any `pandas.Series` with an index including `model` and `scenario`.

In the example, we can now easily derive the "overshoot", i.e., the reduction in global temperature after the peak,
by computing the difference between the two metadata indicators.

In [ ]:
overshoot = df.meta[peak] - df.meta[eoc]

In [ ]:
df.set_meta(name='Overshoot', meta=overshoot)

As a last step of the illustration, we show the first rows of the `meta` dataframe
downselected to the scenarios in the Temperature-category `Below 2.0C`.
This table now includes the three new metadata indicators.

In [ ]:
df.filter(Temperature='Below 2.0C').meta.head()

## Exporting timeseries data for further analysis

The `IamDataFrame` can be exported to `xlsx` and `csv` in the IAMC (wide) format. When writing to `xlsx`, both the timeseries data and the `meta` dataframe of categorization and quantitative indicators will be written to the file,
to two sheets named `data` and `meta` respectively.

This feature is based on [pd.DataFrame.to_excel()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html) and [pd.DataFrame.to_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html). It can use any keyword arguments of that function.

In [ ]:
df.to_excel('tutorial_export.xlsx')